<a href="https://colab.research.google.com/github/juramirezmon/Senales_y_sistemas/blob/main/Parcial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Parcial 2 Señales y Sistemas 2025-1**
#**Juan Esteban Ramírez montoya**


# Punto 1

# Punto 2